# Multi-Factor Prediction of Mental Illness Incidence Rates
**Quinn Bischoff, Eric Matteucci, Rajat Singh, Daniel Velasco**

# Phase II

## Introduction

Behavioural and emotional well-being is integral to the development of societies around the world. However, the rates of incidence of mental health disorders are on the rise in some places around the globe, while others are declining. Our goal is to predict these incidence rates using linear regression and deep learning methods on a rich data set.

Using a combination of data sets that include news headlines, financial indicators, and population distributions and indices, to generate a prediction of incidence of disorders such as depression or anxiety, and deaths by mental health. To this end, news headlines that originate from a given country will be preprocessed using natural language processing (NLP)—specifically, sentiment analysis. The score, in conjunction with the aforementioned datasets, will be used to generate a linear regression model and a neural network. The ultimate goal of this project is to determine whether news headline sentiments from a country are accurate at predicting the mental health disorder rate of the country.

## Phase II Goal
The aim of this phase is to ...

[X] methods will be employed:
1. First
2. ...
X. Last


#### Imports

In [10]:
from functools import reduce

from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor, ridge
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from textblob import TextBlob

import matplotlib.pyplot as plt
import matplotlib.gridspec as grid
import nltk
import numpy as np
import os
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore')

#### Constants

In [11]:
# column names
COL_COUNTRY = 'country'
COL_GDP = 'GDP'
COL_HDI = 'HDI'
COL_POLARITY = 'polarity'
COL_POPULATION_DENSITY = 'population density'
COL_SUBJECTIVITY = 'subjectivity'
COL_UNEMPLOYMENT = 'unemployment rate'
COL_URBAN_DENSITY = 'urban density (%)'
COL_YEAR = 'year'
COL_NEWS_TEMPLATE = 'news{}'

TARGETS = [
    'Bipolar disorder (%)',
    'Eating disorders (%)',
    'Anxiety disorders (%)',
    'Drug use disorders (%)',
    'Depression (%)',
    'Alcohol use disorders (%)',
]

FEATURES = [
    COL_GDP,
    COL_HDI,
    COL_POLARITY,
    COL_POPULATION_DENSITY,
    COL_SUBJECTIVITY,
    COL_UNEMPLOYMENT,
    COL_URBAN_DENSITY,
]

# directories and filenames
DIR_DATA = 'data'
DIR_FINANCIAL = os.path.join(DIR_DATA, 'financial')
DIR_HDI = os.path.join(DIR_DATA, 'human_development_index')
DIR_MENTAL_HEALTH = os.path.join(DIR_DATA, 'mental_health')
DIR_NEWS = os.path.join(DIR_DATA, 'news')
DIR_POPULATION = os.path.join(DIR_DATA, 'population')

FILENAME_DISORDERS = 'prevalence-by-mental-and-substance-use-disorder.csv'
FILENAME_GDP = 'wrldbnk_gdp.csv'
FILENAME_HDI = 'hdi.csv'
FILENAME_NEWS_HEADLINES = 'news_headlines.csv'
FILENAME_POPULATION_DENSITY = 'wrldbnk_pop_dnst.csv'
FILENAME_UNEMPLOYMENT = 'wrldbnk_unemployment.csv'
FILENAME_URBAN_DENSITY = 'wrldbnk_urban_pop.csv'

# Years
DATE_START = '2005'
DATE_END = '2018'

DATE_RANGE = [str(i) for i in range(2005, 2018)]
DATE_NEWS_FROM = ['{}-02-02', '{}-05-05', '{}-07-07', '{}-11-11']
DATE_NEWS_TO = ['{}-03-03', '{}-06-06','{}-08-08', '{}-12-12']


# country values
SELECTED_COUNTRIES = [
    'south africa',
    'kenya',
    'china',
    'taiwan',
    'japan',
    'south korea',
    'india',
    'pakistan',
    'indonesia',
    'philippines',
    'singapore',
    'thailand',
    'canada',
    'united kingdom',
    'ireland',
    'scotland',
    'australia',
    'new zealand',
    'united states',
]


COUNTRIES_DICT = {
    'australia' : 'australia',
    'canada' : 'canada',
    'china' : 'asia/china',
    'india' : 'asia/india',
    'indonesia' : 'asia/southeast/indonesia',
    'ireland' : 'europe/ireland',
    'japan' : 'asia/japan',
    'kenya' : 'africa/kenya',
    'new zealand' : 'new_zealand',
    'pakistan' : 'asia/pakistan',
    'philippines' : 'asia/philippines',
    'scotland' : 'europe/scotland',
    'singapore' : 'asia/singapore',
    'south africa' : 'africa/south_africa',
    'south korea' : 'asia/south_korea',
    'taiwan' : 'asia/taiwan',
    'thailand' : 'asia/thailand',
    'united kingdom' : 'europe/uk',
    'united states' : 'us'
}


# News URL
URL = 'https://newslookup.com/{}?&ut={}&l=1&utto={}'
FOLDS = 5

## Headline Sentiment Data
Headlines were collected for a set of countries by year. In order to use these headlines in further analysis, we want to calculate each headline's polarity and subjectivity and determine a mean for the year.

Headlines are organized in a csv file with the following column headers:

    | year | country | news0 | news1 | news2 | ... | news199 |

Together, the year and country columns are used as the index for the data.
The analysis below creates three output dataframes, one for each headline's polarity, one for each headline's subjectivity, and one with average values of polarity and subjectivity for each year, per country. The headers of each of these are listed below.

polarity_df and subjectivity_df:

    | year | country | news0 | news1 | news2 | ... | news199 |

average_sentiment_df:

    | year | country | polarity | subjectivity |

Both of the dataframes are indexed by year and country.


In [12]:
# Read in the parsed news headlines from the csv file
file_path = os.path.join(DIR_NEWS, FILENAME_NEWS_HEADLINES)
headlines_df = pd.read_csv(file_path, sep='|', index_col=(0, 1))

average_sentiment_columns = [COL_YEAR, COL_COUNTRY, COL_POLARITY, COL_SUBJECTIVITY]
average_sentiment_df = pd.DataFrame(columns=average_sentiment_columns)

news_columns = [COL_NEWS_TEMPLATE.format(i) for i in range(200)]
news_columns.insert(0, COL_COUNTRY)
news_columns.insert(0, COL_YEAR)
polarity_df = pd.DataFrame(columns=news_columns)
subjectivity_df = pd.DataFrame(columns=news_columns)

# iterate through the headline rows
for index, row in headlines_df.iterrows():
    # lists to store the individual values for each headline
    polarity_list = list()
    subjectivity_list = list()

    polarity_list.extend([index[0], index[1]])
    subjectivity_list.extend([index[0], index[1]])

    # values for the avgerage yearly polarity and subjectivity
    yearly_average_polarity = 0
    yearly_average_subjectivity = 0
    yearly_average_count = 0

    # calculate polarity and subjectivity for each headline
    for entry in row:
        if type(entry) == float:
            polarity_list.append(entry)
            subjectivity_list.append(entry)

        else:
            blob = TextBlob(entry)

            pol_val = 0
            sub_val = 0
            count = 0

            # average the values in case a headline is multiple sentences
            for sentence in blob.sentences:
                pol_val = pol_val + sentence.sentiment.polarity
                sub_val = sub_val + sentence.sentiment.subjectivity
                count = count + 1

            polarity_list.append(pol_val / count)
            subjectivity_list.append(sub_val / count)

            yearly_average_polarity = yearly_average_polarity + pol_val / count
            yearly_average_subjectivity = yearly_average_subjectivity + sub_val / count
            yearly_average_count = yearly_average_count + 1

    yearly_average_polarity = yearly_average_polarity / yearly_average_count
    yearly_average_subjectivity = yearly_average_subjectivity / yearly_average_count

    yearly_average_df = pd.DataFrame([[index[0], index[1], yearly_average_polarity, yearly_average_subjectivity]], columns=average_sentiment_columns)

    pol_row_df = pd.DataFrame([polarity_list], columns=news_columns)
    sub_row_df = pd.DataFrame([subjectivity_list], columns=news_columns)

    average_sentiment_df = pd.concat([average_sentiment_df, yearly_average_df], sort=False)
    polarity_df = pd.concat([polarity_df, pol_row_df], sort=False)
    subjectivity_df = pd.concat([subjectivity_df, sub_row_df], sort=False)

# these are all of the polarities and subjectivities for each headline
polarity_df = polarity_df.set_index([COL_YEAR, COL_COUNTRY])
subjectivity_df = subjectivity_df.set_index([COL_YEAR, COL_COUNTRY])
average_sentiment_df = average_sentiment_df.set_index([COL_YEAR, COL_COUNTRY])


## Data Loading

In [13]:
def load_data_frame(file_name, path, sep=None):
    """
    Loads data from specified path and name, returns a dataframe
    """
    file_path = os.path.join(path, file_name)
    if not sep:
        return pd.read_csv(file_path)
    return pd.read_csv(file_path, sep=sep)


def load_mental_health_data():
    """
    Loads mental health data, and performs basic preprocessing operations:
        - columns renamed appropriately for compatibility
        - selected countries are filtered
        - unnecessary columns are dropped
    """
    mental_df = load_data_frame(
        FILENAME_DISORDERS,
        DIR_MENTAL_HEALTH
    )

    mental_df.rename(columns={'Entity': COL_COUNTRY, 'Year': COL_YEAR}, inplace=True)
    mental_df.drop(labels='Code', axis=1, inplace=True)
    mental_df[COL_COUNTRY] = mental_df[COL_COUNTRY].str.lower()
    mental_df = mental_df[mental_df[COL_COUNTRY].isin(SELECTED_COUNTRIES)]
    return mental_df


def load_world_bank_data(filename, directory, value):
    """
    Loads World Bank data, and performs basic preprocessing operations:
        - columns renamed appropriately for compatibility
        - selected countries are filtered
        - unnecessary columns are dropped
        - non-numerical values are replaced with NaN or transformed appropriately
    """
    df = load_data_frame(
        filename,
        directory
    )
    df.rename(columns={'Country Name': COL_COUNTRY}, inplace=True)
    df.drop(['Indicator Name', 'Indicator Code', 'Country Code'], axis=1, inplace = True)
    df = df.replace('..', np.NaN)
    df.loc[:,1:] = df.iloc[:, 1:].apply(pd.to_numeric)
    df[COL_COUNTRY] = df[COL_COUNTRY].str.lower()
    df = df.replace('korea, rep.', 'south korea')
    df = df.loc[df[COL_COUNTRY].isin(SELECTED_COUNTRIES)]

    columns = df.columns
    country_index = columns.get_loc(COL_COUNTRY)
    start_index = columns.get_loc(DATE_START)
    end_index = columns.get_loc(DATE_END)
    df = df.iloc[:, np.r_[country_index, start_index:end_index]]

    # Reshape the dataframe to structure `country|year|value`
    df = pd.melt(
        df,
        id_vars=COL_COUNTRY,
        var_name=COL_YEAR,
        value_name=value
    )
    df[COL_YEAR] = df[COL_YEAR].apply(pd.to_numeric)
    df = df.sort_values([COL_COUNTRY, COL_YEAR])
    df.reset_index(inplace=True, drop=True)
    df.dropna(axis=0, inplace=True)
    return df


def load_hdi_data():
    """
    Loads human development index (HDI) data, and performs basic preprocessing operations:
        - columns are renamed for compatibility
        - selected countries are filtered
        - unnecessary columns are dropped
    """
    hdi_df = load_data_frame(
        FILENAME_HDI,
        DIR_HDI
    )

    hdi_df = hdi_df.dropna(how='all', axis=1)
    hdi_df.drop('HDI Rank (2017)', axis=1, inplace=True)
    hdi_df.rename(columns={'Country': COL_COUNTRY, 'Year': COL_YEAR}, inplace=True)
    hdi_df[COL_COUNTRY] = hdi_df[COL_COUNTRY].str.lower().str.strip()
    hdi_df = hdi_df[hdi_df[COL_COUNTRY].isin(SELECTED_COUNTRIES)]
    hdi_df = hdi_df.reset_index(drop=True)
    hdi_df.loc[:,1:] = hdi_df.iloc[:,1:].apply(pd.to_numeric)

    # Reshape the dataframe to structure `country|year|value`
    hdi_df = pd.melt(
        hdi_df,
        id_vars=[COL_COUNTRY],
        var_name=COL_YEAR,
        value_name=COL_HDI
    )

    hdi_df[COL_YEAR] = hdi_df[COL_YEAR].apply(pd.to_numeric)
    hdi_df = hdi_df.drop(hdi_df[hdi_df.year < int(DATE_START)].index)
    hdi_df = hdi_df.drop(hdi_df[hdi_df.year > int(DATE_END)].index)
    return hdi_df



## Joining Datasets

In [14]:
population_df = load_world_bank_data(FILENAME_POPULATION_DENSITY, DIR_POPULATION, COL_POPULATION_DENSITY)
urban_df = load_world_bank_data(FILENAME_URBAN_DENSITY, DIR_POPULATION, COL_URBAN_DENSITY)
gdp_df = load_world_bank_data(FILENAME_GDP, DIR_FINANCIAL, COL_GDP)
unemployment_df = load_world_bank_data(FILENAME_UNEMPLOYMENT, DIR_FINANCIAL, COL_UNEMPLOYMENT)
hdi_df = load_hdi_data()
mental_df = load_mental_health_data()



# Joining individual datasets on population and urban population density, 
# GDP, unemployment, HDI, average news headlines sentiment, and mental health data
data_frames_list = [
    population_df,
    urban_df,
    gdp_df,
    unemployment_df,
    hdi_df,
    average_sentiment_df, 
    mental_df
]

joined_df = reduce(lambda left, right: pd.merge(left, right, on=[COL_COUNTRY, COL_YEAR], how='inner'), data_frames_list)


## Pipeline Setup

In [34]:
# Regressors and their tuning parameters
net = MLPRegressor()
net_parameters = {
    'net__learning_rate': ['constant', 'invscaling', 'adaptive'],
    'net__hidden_layer_sizes': [(50), (100), (50, 50), (100, 100), (150, 150), (100, 100, 100)],
    'net__max_iter': [2, 10, 100, 1000]
}

linear = LinearRegression()
linear_parameters = {
    'linear__fit_intercept': [True, False]
}

forest = RandomForestRegressor()
forest_parameters = {
    'forest__n_estimators': [10, 20, 50, 70, 100],
    'forest__max_depth': [2, 5, 10],
    'forest__min_samples_split': [2, 3, 4, 5],
    'forest__max_features': ['auto', 'sqrt', 'log2']
}

stochastic = SGDRegressor()
stochastic_parameters = {
    'stochastic__alpha': [0.0001, 0.001, 0.01, 0.1],
    'stochastic__penalty': ['l2','l1'],
    'stochastic__learning_rate': ['invscaling', 'optimal', 'constant', 'adaptive']
}

knn = KNeighborsRegressor()
knn_parameters = {
    'knn__n_neighbors': [5, 10, 20, 30]
}

# Scalers
scaler = MinMaxScaler()

# Feature Selectors
feature_selector = SelectKBest()
selector_parameters = {
    'selector__k': ['all', 1, 2, 3, 4, 5, 6]
}

# Pipelines
net_pipeline = Pipeline(steps=[('scaler', scaler), ('selector', feature_selector), ('net', net)])
linear_pipeline = Pipeline(steps=[('scaler', scaler), ('selector', feature_selector), ('linear', linear)])
forest_pipeline = Pipeline(steps=[('scaler', scaler), ('selector', feature_selector), ('forest', forest)])
stochastic_pipeline = Pipeline(steps=[('scaler', scaler), ('selector', feature_selector), ('stochastic', stochastic)])
knn_pipeline = Pipeline(steps=[('scaler', scaler), ('selector', feature_selector), ('knn', knn)])


def get_grid_search(pipeline, parameters):
    return GridSearchCV(pipeline, parameters, cv=FOLDS, n_jobs=-1, verbose=5)
    

grid_searcher_net = get_grid_search(net_pipeline, dict(net_parameters, **selector_parameters))
# grid_searcher_linear = get_grid_search(linear_pipeline, dict(selector_parameters, **linear_parameters))
# grid_searcher_forest = get_grid_search(forest_pipeline, dict(selector_parameters, **forest_parameters))
# grid_search_stochastic = get_grid_search(stochastic_pipeline, dict(selector_parameters, **stochastic_parameters))
# grid_searcher_knn = get_grid_search(knn_pipeline, dict(selector_parameters, **knn_parameters))


In [35]:
joined_df.columns

Index(['country', 'year', 'population density', 'urban density (%)', 'GDP',
       'unemployment rate', 'HDI', 'polarity', 'subjectivity',
       'Schizophrenia (%)', 'Bipolar disorder (%)', 'Eating disorders (%)',
       'Anxiety disorders (%)', 'Drug use disorders (%)', 'Depression (%)',
       'Alcohol use disorders (%)'],
      dtype='object')

<b>Anxiety Disorders</b>

In [36]:
# Removing indexing columns (country, year) and rates that will not be predicted
TARGET = 'Anxiety disorders (%)'

y = joined_df.copy().pop(TARGET)
x = joined_df.copy()[FEATURES]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
grid_searcher_net.fit(x_train, y_train)

Fitting 5 folds for each of 504 candidates, totalling 2520 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 295 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 609 tasks      | elapsed:   59.5s
[Parallel(n_jobs=-1)]: Done 967 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1326 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1748 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 2239 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 2513 out of 2520 | elapsed:  3.1min remaining:    0.5s
[Parallel(n_jobs=-1)]: Done 2520 out of 2520 | elapsed:  3.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selector', SelectKBest(k=10, score_func=<function f_classif at 0x11bf967b8>)), ('net', MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hid...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'net__learning_rate': ['constant', 'invscaling', 'adaptive'], 'net__hidden_layer_sizes': [50, 100, (50, 50), (100, 100), (150, 150), (100, 100, 100)], 'net__max_iter': [2, 10, 100, 1000], 'selector__k': ['all', 1, 2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [37]:
print('Best estimator:', grid_searcher_net.best_estimator_)
print('Best score:', grid_searcher_net.best_score_)
print('Best parameters:', grid_searcher_net.best_params_)

net = grid_searcher_net.classifier.best_estimator_.named_steps.net

if not self.features:
    num_features_used = self.classifier.best_params_['selector__k']
    scores = self.classifier.best_estimator_.named_steps.selector.scores_
    highest_scores_indices = numpy.argpartition(scores, -num_features_used)[-num_features_used:]

    features_used = x_train.columns[highest_scores_indices].values

Best estimator: Pipeline(memory=None,
     steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('selector', SelectKBest(k=6, score_func=<function f_classif at 0x11bf967b8>)), ('net', MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidd...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))])
Best score: 0.8154689346302422
Best parameters: {'net__hidden_layer_sizes': (50, 50), 'net__learning_rate': 'constant', 'net__max_iter': 1000, 'selector__k': 6}
